In [0]:
%run ../_utils

In [0]:
from pyspark.sql.functions import col, count, when


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar os meses em que tiveram mais pedidos

In [0]:
tb_name = "olist_gold.multiple_orders"
dataset_location = "olist_multiple_orders_dataset"
target_location = f"dbfs:/FileStore/delta/brazilian_ecommerce/{dataset_location}/gold"

## 1 - Data ingestion


In [0]:
df = spark.read.table("olist_gold.customer_orders") # leituira da delta table central, orders (ja da propria gold)


## 2 - preparation

In [0]:
df = df.groupBy("customer_unique_id").agg(
    count("order_id").alias("total_orders"),
)


# 2.1 multiple_orders?

informar se o cliente comprou mais de uma vez

In [0]:
df = df.withColumn("multiple_order", when(col("total_orders") > 1, True).otherwise(False))

In [0]:
display(df.take(10))


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)


## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from olist_gold.multiple_orders limit 10